In [2]:
import pyvista as pv
import numpy as np
from pathlib import Path

In [3]:
file = Path('blood/centerlines.vtp')
cl = pv.read(file)

In [4]:
cl.n_cells, cl.n_lines, cl.n_arrays, cl.n_points

(6, 6, 1, 701)

In [5]:
cl2 = cl.compute_arc_length()
cl2['arc_length'].shape

(701,)

In [6]:
cl.head()

'PolyData (0x11cd2c8e0)\n  N Cells:\t6\n  N Points:\t701\n  X Bounds:\t-4.257e+01, 2.567e+01\n  Y Bounds:\t-1.400e+00, 4.615e+01\n  Z Bounds:\t-1.812e+01, 9.256e+01\n  N Arrays:\t1\n'

In [23]:
npoints = [cl.cell_n_points(i) for i in range(cl.n_cells)]
start = np.cumsum([0]+npoints)
lines = [np.arange(start[i],start[i+1]) for i in range(len(start)-1)]
cl.compute_arc_length()
cl.cell_data

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    vtkOriginalCellIds      int64    (6,)

In [18]:
cl2 = cl.extract_cells(5)
cl2[cl2.array_names[1]]

array([574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586,
       587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599,
       600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612,
       613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625,
       626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638,
       639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651,
       652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664,
       665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677,
       678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690,
       691, 692, 693, 694, 695, 696, 697, 698, 699, 700])

In [9]:
lines = [pv.PolyData(cl.cell_points(i))  for i in range(cl.n_cells)]
#length = [l['arc_length'] for l in lines]

In [10]:
lines[0]

PolyData,Information
N Cells,89
N Points,89
X Bounds,"-2.838e+01, -1.971e+01"
Y Bounds,"2.862e+01, 3.915e+01"
Z Bounds,"-1.812e+01, 5.463e+01"
N Arrays,0
